<a href="https://www.kaggle.com/code/samithsachidanandan/functional-api-tf-mnist?scriptVersionId=277941357" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Importing the libraries 

In [1]:
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-me

In [2]:
import tensorflow as tf
from tensorflow import keras 
import numpy as np 
import matplotlib.pyplot as plt 

2025-11-14 16:33:19.347951: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763137999.592026      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763137999.659110      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Define Functionsl Model 

In [3]:
inputs = keras.Input(shape=(28,28))
flatten = keras.layers.Flatten()
dense1 = keras.layers.Dense(128, activation='relu')

dense2 = keras.layers.Dense(10, activation='softmax', name='category_output')
dense3 = keras.layers.Dense(1, activation='sigmoid', name='leftright_output')

In [4]:
x = flatten(inputs)
x = dense1(x)
output1 = dense2(x)
output2 = dense3(x)

model = keras.Model(inputs= inputs, outputs=[output1, output2], name="mnist_model")

2025-11-14 16:33:39.291663: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
model.summary()

Model: "mnist_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 28, 28)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 784)       │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │    100,480 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ category_output     │ (None, 10)        │      1,290 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leftright_output    │ (None, 1)         │        129 │ dense[0][0]       │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 101,899 (398.04 KB)

 Trainable params: 101,899 (398.04 KB)

 Non-trainable params: 0 (0.00 B)

### Loss and Optimizer 

In [6]:
loss1 = keras.losses.SparseCategoricalCrossentropy(from_logits= False)
loss2 = keras.losses.BinaryCrossentropy(from_logits= False)
optim = keras.optimizers.Adam(learning_rate = 0.001)
metrics= ["accuracy"]

losses = {
    "category_output": loss1,
    "leftright_output" : loss2
   
}

model.compile(
    loss=losses,
    optimizer=optim,
    metrics={
        "category_output": "accuracy",
        "leftright_output": "accuracy"
    }
)

### Create data with 2 labels 

In [7]:
mnist = keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

# 0=left, 1=right
y_leftright = np.zeros(y_train.shape, dtype=np.uint8)
for idx, y in enumerate(y_train):
    if y>5:
        y_leftright[idx] = 1


print(y_train.dtype, y_train[0:20])
print(y_leftright.dtype, y_leftright[0:20])

y= {"category_output": y_train,
   "leftright_output" : y_leftright}


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
uint8 [5 0 4 1 9 2 1 3 1 4 3 5 3 6 1 7 2 8 6 9]
uint8 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 1 1]


### Training

In [8]:
model.fit(X_train, y=y, epochs=5,
         batch_size=64, verbose=2)

Epoch 1/5
938/938 - 5s - 5ms/step - category_output_accuracy: 0.9137 - category_output_loss: 0.3091 - leftright_output_accuracy: 0.9310 - leftright_output_loss: 0.1909 - loss: 0.5002
Epoch 2/5
938/938 - 3s - 3ms/step - category_output_accuracy: 0.9591 - category_output_loss: 0.1396 - leftright_output_accuracy: 0.9675 - leftright_output_loss: 0.0954 - loss: 0.2351
Epoch 3/5
938/938 - 3s - 3ms/step - category_output_accuracy: 0.9713 - category_output_loss: 0.0989 - leftright_output_accuracy: 0.9749 - leftright_output_loss: 0.0745 - loss: 0.1735
Epoch 4/5
938/938 - 3s - 3ms/step - category_output_accuracy: 0.9776 - category_output_loss: 0.0762 - leftright_output_accuracy: 0.9792 - leftright_output_loss: 0.0614 - loss: 0.1376
Epoch 5/5
938/938 - 3s - 3ms/step - category_output_accuracy: 0.9818 - category_output_loss: 0.0618 - leftright_output_accuracy: 0.9815 - leftright_output_loss: 0.0529 - loss: 0.1148


### list with 2 predictions

In [9]:
predictions = model.predict(X_test)
len(predictions)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


2

In [10]:
prediction_category = predictions[0]
prediction_lr = predictions[1]

pr_cat = prediction_category[0:20]
prediction_lr = prediction_lr[0:20]

labels_cat = np.argmax(pr_cat, axis =1)
labels_lr= np.array([1 if p>=0.5 else  0 for p in prediction_lr])

In [11]:
print(y_test[0:20])
print(labels_cat)
print(labels_lr)

[7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
[7 2 1 0 4 1 4 9 6 9 0 6 9 0 1 5 9 7 3 4]
[1 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 0]
